라이브러리

In [10]:
from tensorflow.keras.datasets import mnist
import numpy as np

1. 미니배치 학습 (Mini-Batch Learning)
    - 전체 데이터를 한 번에 처리하는 대신, 일부 데이터만 골라서 학습하는 방법.

- 1.1 전체 오차 함수 (Loss)
    - 훈련 데이터가 $N$개일 때, 전체 오차 함수는 다음과 같다.
        - 수식 : $E = -\frac{1}{N} \sum_{n=1}^{N} \sum_{k=1}^{C} t_{nk} \log(y_{nk})$
            - $t_{nk}$ : 정답 레이블 (one-hot encoding)4
            - $y_nk$ : 신경망 출력 (softmax 확률)
            - $N$ : 데이터 개수


- 1.2 문제점
    - 전체 데이터로 손실을 계산하면 시간이 너무 오래 걸린다.
    - 따라서 일부 샘플만 뽑아 손실을 근사 계산하는 미니배치 방법을 사용한다.



2. MNIST 데이터셋 불러오기

In [8]:
# 데이터 다운로드 및 로드
(x_train, t_train), (x_test, t_test) = mnist.load_data()

# 데이터 형태 출력
print(x_train.shape)  # (60000, 28, 28)
print(t_train.shape)  # (60000,)
print(x_test.shape)   # (10000, 28, 28)
print(t_test.shape)   # (10000,)


(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


3. 미니배치 만들기

In [9]:
train_size = x_train.shape[0]
batch_size = 10  # 미니배치 크기

# 무작위로 10개 샘플을 선택
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]


4. 교차 엔트로피 오차 (Cross Entropy Error)

- 4.1 교차 엔트로피 함수 구현

In [11]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

- 4.2 설명
    - y: 신경망 출력 결과 (softmax 확률)
    - t: 정답 레이블 (0~9 숫자)
    - y[np.arange(batch_size), t]: 정답 레이블에 해당하는 확률만 선택
    - log 값을 취하고 평균을 내서 최종 손실값 계산

- 4.3 주의
    - 입력 y가 1차원인 경우, (ex: 하나의 데이터만 들어온 경우) 2차원으로 변환해줘야 한다.



5. 손실 함수를 설정하는 이유
    - 신경망 학습 시 "정확도(Accuracy)"를 바로 최적화 대상으로 삼으면 안 된다.
        - 이유
            - 정확도는 이산적(discrete)으로 변한다.        
            - 매개변수를 아주 조금씩 바꿔도 정확도가 거의 변하지 않아서, 미분이 대부분 0이 된다.
            - 따라서 학습이 전혀 일어나지 않게 된다.